In [1]:
#just loadingblast and seeing if it is correctly installed  

from Bio.Blast import NCBIWWW
help(NCBIWWW.qblast)

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None, username='blast', password=No

Check if Protein names are valid

In [7]:
from Bio import Entrez
import pandas as pd

# Load Excel file and extract gene names and species
df_path = "Dros_gene_names_full.xlsx"
df = pd.read_excel(df_path, sheet_name='Dros_gene_names_full', header=0)

# Drop rows with any NaN values
df = df.dropna()

# Set email for NCBI
Entrez.email = "albertothornton@outlook.com"

# Function to print details for genes with no protein sequences found
def print_missing_protein_sequences(df):
    for index, row in df.iterrows():
        gene_ID = row['Gene ID']
        species = row['Species']  # Assume 'Species' is the column name for species information

        try:
            # Search for the protein related to the gene in the specified species
            query = f"{gene_ID}[Gene] AND {species}[Organism]"
            search_handle = Entrez.esearch(db="protein", term=query)
            search_record = Entrez.read(search_handle)
            search_handle.close()

            # Print gene-species pairs where no protein sequences are found
            if not search_record["IdList"]:
                # Print the full row details for missing protein sequences
                print(f"Missing protein sequence for: {row.to_dict()}")
        except Exception as e:
            print(f"Error processing {gene_ID} in {species}: {str(e)}")

# Run the function to print missing proteins
print_missing_protein_sequences(df)


Missing protein sequence for: {'Gene name': 'Test1234545', 'Gene ID': 'testtestest', 'Category': 'test test. Test', 'Species': 'Drosophila melanogaster'}


Load and Download all genes from NCBI library 

In [ ]:
import os  # For directories
from Bio import Entrez  # For accessing the NCBI database
import pandas as pd  # For handling the Excel file
import time  # To add delays between requests

# Load Excel file and extract gene names, species, and IDs
df_path = "Dros_gene_names_full.xlsx"  # Update with the correct file name if needed
sheet_name = 'Dros_gene_names_full'  # Specify the sheet name

df = pd.read_excel(df_path, sheet_name=0)

# Drop rows with any missing values
df = df.dropna()

# Set email for NCBI
Entrez.email = "albertothornton@outlook.com"  # Please enter your email

# Define output directory for the downloaded files
output_dir = "Protein_FASTA_Files"  # You can change this name; this is just an example
os.makedirs(output_dir, exist_ok=True)  # Creates the directory if it does not exist

# Function to clear the console (works for most environments)
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')

# Function to download protein sequences grouped by Gene Name and save species-specific combined files
def download_and_combine_species_files(df):
    not_found_proteins = []  # List to track protein sequences that were not found

    for gene_name, group in df.groupby('Gene name'):  # Define the column with the general gene names
        # Replace spaces and special characters in gene names for directory names; this is so the names can be easily used later in the BLAST code
        safe_gene_name = gene_name.replace(" ", "_").replace("/", "_")
        
        # Create a directory for the current Gene Name
        gene_dir = os.path.join(output_dir, safe_gene_name)
        os.makedirs(gene_dir, exist_ok=True)

        for species, species_group in group.groupby('Species'):  # Specifies species orthologs to be downloaded based on Species column
            # Replace spaces and special characters in species names for file names
            safe_species_name = species.replace(" ", "_").replace("/", "_")  # Sanitize gene name
            
            # Path for the combined FASTA file for the current species
            combined_fasta_path = os.path.join(gene_dir, f"{safe_species_name}_combined.fasta")  # Output file path
            
            try:
                # Print a message for current processing
                print(f"Processing: Gene={safe_gene_name}, Species={safe_species_name}")

                with open(combined_fasta_path, "w") as combined_fasta_file:  # Open file to write the sequence
                    for index, row in species_group.iterrows():
                        gene_id = row['Gene ID']  # Change this with Gene ID column

                        # Retry mechanism for NCBI requests since sometimes it times out
                        retries = 3
                        for attempt in range(retries):
                            try:
                                # Search for the protein related to the gene in the specified species
                                search_term = f"{gene_id}[Gene] AND {species}[Organism]"
                                search_handle = Entrez.esearch(db="protein", term=search_term)
                                search_record = Entrez.read(search_handle)
                                search_handle.close()

                                # Check if any protein sequences were found
                                if search_record["IdList"]:
                                    for sequence_id in search_record["IdList"]:
                                        fetch_handle = Entrez.efetch(db="protein", id=sequence_id, rettype="fasta", retmode="text")
                                        sequence_data = fetch_handle.read()
                                        fetch_handle.close()

                                        # Write sequence data to the combined file
                                        combined_fasta_file.write(sequence_data)
                                    break  # Exit retry loop if successful
                                else:
                                    # Add to not-found list
                                    not_found_proteins.append((gene_id, species))
                                    print(f"No protein sequences found for {gene_id} in {species}")
                                    break  # Exit retry loop if no sequences are found
                            except Exception as e:
                                if attempt < retries - 1:
                                    print(f"Retrying ({attempt + 1}/{retries}) for {gene_id} in {species}...")
                                    time.sleep(5)  # Wait 5 seconds before retrying to avoid timing out
                                else:
                                    print(f"Failed after {retries} attempts for {gene_id} in {species}")
                                    raise e  # Raise the exception after all retries fail
            except Exception as e:
                print(f"Error retrieving or combining data for {species} in gene {gene_name}: {e}")  # In case sequences were not found

    # Clear console before printing the summary
    clear_console()

    # Print all proteins not found
    print("\nSummary: Proteins Not Found")
    if not_found_proteins:
        for gene_id, species in not_found_proteins:
            print(f"Gene ID: {gene_id}, Species: {species}")
    else:
        print("All proteins were successfully retrieved!")

    print("All genes and species processed successfully!")

# Run the function to download and combine protein sequences into species-specific files for each gene group
download_and_combine_species_files(df)


Processing: Gene=Ank2_(Ankyrin_2), Species=Drosophila_ananassae
Processing: Gene=Ank2_(Ankyrin_2), Species=Drosophila_erecta
Processing: Gene=Ank2_(Ankyrin_2), Species=Drosophila_mauritiana
Processing: Gene=Ank2_(Ankyrin_2), Species=Drosophila_melanogaster
Processing: Gene=Ank2_(Ankyrin_2), Species=Drosophila_simulans
Processing: Gene=Ank2_(Ankyrin_2), Species=Drosophila_teissieri
Processing: Gene=Ank2_(Ankyrin_2), Species=Drosophila_yakuba
Processing: Gene=Arr2_(Arrestin_2), Species=Drosophila_ananassae
Processing: Gene=Arr2_(Arrestin_2), Species=Drosophila_erecta
Processing: Gene=Arr2_(Arrestin_2), Species=Drosophila_mauritiana
Processing: Gene=Arr2_(Arrestin_2), Species=Drosophila_melanogaster
Processing: Gene=Arr2_(Arrestin_2), Species=Drosophila_simulans
Processing: Gene=Arr2_(Arrestin_2), Species=Drosophila_teissieri
Processing: Gene=Arr2_(Arrestin_2), Species=Drosophila_yakuba
Processing: Gene=Bmcp, Species=Drosophila_ananassae
Processing: Gene=Bmcp, Species=Drosophila_erecta
Pr

Create a BLAST database

In [ ]:
import os  # For directory management
import subprocess  # For running external commands

# Define the base folder
base_dir = "Protein_FASTA_Files"  # Folder containing gene subfolders
db_dir = "blast_databases"  # Root folder to store BLAST databases

# Create the root database folder if it doesn't exist
os.makedirs(db_dir, exist_ok=True)

# Function to clean and validate FASTA files
def clean_fasta(input_file):
    """Cleans and validates a FASTA file by simplifying the header and removing invalid characters."""
    with open(input_file, "r") as infile:
        lines = infile.readlines()

    with open(input_file, "w") as outfile:  # Overwrite the original file
        for line in lines:
            if line.startswith(">"):
                # Retain only the sequence ID from the header
                sequence_id = line.split()[0]  # Keep everything before the first space
                outfile.write(sequence_id + "\n")  # Write the cleaned header
            else:
                # Remove spaces and write the cleaned sequence
                outfile.write(line.strip() + "\n")

# Function to clear the console (works for most environments)
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')

# List to track failed database creations
failed_dbs = []

# Loop through the gene folders to create BLAST databases
for gene_folder in os.listdir(base_dir):
    gene_path = os.path.join(base_dir, gene_folder)  # Path to each gene folder
    if os.path.isdir(gene_path):  # Only process folders
        # Create a separate subfolder in `blast_databases` for each gene
        gene_db_folder = os.path.join(db_dir, gene_folder)
        os.makedirs(gene_db_folder, exist_ok=True)

        # Process each FASTA file within the gene folder
        for fasta_file in os.listdir(gene_path):
            if fasta_file.endswith(".fasta"):
                fasta_path = os.path.join(gene_path, fasta_file)  # Path to the FASTA file

                # Clean the FASTA file (overwrite the original file)
                clean_fasta(fasta_path)

                # Create a BLAST database in the gene-specific folder
                db_name = os.path.join(gene_db_folder, os.path.splitext(fasta_file)[0])
                try:
                    subprocess.run(
                        ["makeblastdb", "-in", fasta_path, "-dbtype", "prot", "-out", db_name],
                        check=True,
                    )
                except subprocess.CalledProcessError as e:
                    failed_dbs.append((fasta_file, str(e)))

# Clear console before displaying results
clear_console()

# Final message with failed databases
if failed_dbs:
    print("\nFailed Database Creations:")
    for fasta_file, error in failed_dbs:
        print(f"File: {fasta_file}, Error: {error}")
else:
    print("All gene folders processed successfully without errors!")




Building a new DB, current time: 11/19/2024 11:46:21
New DB name:   /Users/albertothornton/Desktop/Kamikouchi lab/Blast Code and data/blast_databases/sosie/Drosophila_melanogaster_combined
New DB title:  Protein_FASTA_Files/sosie/Drosophila_melanogaster_combined.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 14 sequences in 0.00563121 seconds.


Building a new DB, current time: 11/19/2024 11:46:21
New DB name:   /Users/albertothornton/Desktop/Kamikouchi lab/Blast Code and data/blast_databases/sosie/Drosophila_teissieri_combined
New DB title:  Protein_FASTA_Files/sosie/Drosophila_teissieri_combined.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000642061 seconds.


Building a new DB, current time: 11/19/2024 11:46:21
New DB name:   /Users/albertothornton/Desktop/Kamikouchi lab/Blast Code and data/blast_databases/sosie/Drosophila_yakuba_combined
N

BLAST options error: File Protein_FASTA_Files/Eb1/Drosophila_yakuba_combined.fasta is empty




Building a new DB, current time: 11/19/2024 11:46:38
New DB name:   /Users/albertothornton/Desktop/Kamikouchi lab/Blast Code and data/blast_databases/Eb1/Drosophila_erecta_combined
New DB title:  Protein_FASTA_Files/Eb1/Drosophila_erecta_combined.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 5 sequences in 0.00119996 seconds.


Building a new DB, current time: 11/19/2024 11:46:38
New DB name:   /Users/albertothornton/Desktop/Kamikouchi lab/Blast Code and data/blast_databases/Eb1/Drosophila_simulans_combined
New DB title:  Protein_FASTA_Files/Eb1/Drosophila_simulans_combined.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 0.00116301 seconds.


Building a new DB, current time: 11/19/2024 11:46:38
New DB name:   /Users/albertothornton/Desktop/Kamikouchi lab/Blast Code and data/blast_databases/Eb1/Drosophila_mauritiana_combined
New DB title:  Protein_

KeyboardInterrupt: 

Loop for all genes

In [ ]:
import os
import pandas as pd
import subprocess
from io import StringIO

# Paths and parameters
main_gene_folder = "Protein_FASTA_Files"        # Main folder containing gene subfolders
db_folder = "blast_databases"                  # Root folder for BLAST databases
evalue = 0.001                                  # E-value threshold for BLAST
max_hits = 10                                   # Max hits to retain from BLAST results
output_folder = "cross_species_blast_results"  # Folder to store all results
master_csv_file = os.path.join(output_folder, "master_blast_results.csv")  # Master results file

# Create output folder
os.makedirs(output_folder, exist_ok=True)

# Function to sanitize names
def sanitize_name(name):
    """Remove special characters and spaces from a string."""
    return "".join(c for c in name if c.isalnum() or c == "_")

# Function to clear the console (works for most environments)
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')

# Function to run BLAST for each species against all other species
def run_cross_species_blast(main_gene_folder, db_folder, output_folder, evalue, max_hits):
    master_df = pd.DataFrame(columns=[
        "Gene Name", "Query Species", "Target Species", "Query ID", "Subject ID", 
        "% Identity", "Alignment Length", "Mismatches", "Gap Opens", "Query Start", 
        "Query End", "Subject Start", "Subject End", "E-value", "Bit Score"
    ])
    failed_blasts = []  # List to track failed BLAST commands
    
    # Loop through each gene folder
    for gene_folder in os.listdir(main_gene_folder):
        gene_path = os.path.join(main_gene_folder, gene_folder)
        if os.path.isdir(gene_path):  # Check if it's a directory
            print(f"Processing gene: {gene_folder}")
            
            # Create a subfolder for the current gene in the output folder
            gene_output_folder = os.path.join(output_folder, gene_folder)
            os.makedirs(gene_output_folder, exist_ok=True)
            
            # Iterate over species within the gene folder
            species_files = [f for f in os.listdir(gene_path) if f.endswith("_combined.fasta")]
            for query_file in species_files:
                query_species = query_file.split("_combined")[0]
                query_path = os.path.join(gene_path, query_file)
                
                # Prepare an output CSV for the current query species
                query_output_csv = os.path.join(gene_output_folder, f"{query_species}_vs_all.csv")
                query_results_df = pd.DataFrame(columns=[
                    "Query ID", "Subject ID", "% Identity", "Alignment Length", "Mismatches", 
                    "Gap Opens", "Query Start", "Query End", "Subject Start", "Subject End", 
                    "E-value", "Bit Score", "Target Species"
                ])
                
                # BLAST the query species against all other species
                for target_file in species_files:
                    target_species = target_file.split("_combined")[0]
                    if query_species != target_species:  # Avoid self-comparison
                        # Construct the BLAST database name based on the new format
                        target_db_folder = os.path.join(db_folder, gene_folder)
                        target_db = os.path.join(target_db_folder, f"{target_species}_combined")
                        
                        # Check if the BLAST database exists before running
                        if not os.path.exists(f"{target_db}.phr") or not os.path.exists(f"{target_db}.pin") or not os.path.exists(f"{target_db}.psq"):
                            failed_blasts.append((query_species, target_species, "Database not found"))
                            continue
                        
                        # Run BLASTp
                        blast_cmd = [
                            "blastp",
                            "-query", query_path,
                            "-db", target_db,
                            "-evalue", str(evalue),
                            "-outfmt", "6",
                            "-max_target_seqs", str(max_hits)
                        ]
                        try:
                            subprocess.run(blast_cmd, capture_output=True, text=True, check=True)
                        except subprocess.CalledProcessError as e:
                            failed_blasts.append((query_species, target_species, str(e)))
                
                # Append to master DataFrame
                query_results_df["Gene Name"] = gene_folder
                query_results_df["Query Species"] = query_species
                master_df = pd.concat([master_df, query_results_df], ignore_index=True)
    
    # Save master CSV
    master_df.to_csv(master_csv_file, index=False)
    
    # Clear console before printing failures
    clear_console()
    
    # Print any failed BLAST commands
    if failed_blasts:
        print("Failed BLAST Commands:")
        for query_species, target_species, error in failed_blasts:
            print(f"Query Species: {query_species}, Target Species: {target_species}, Error: {error}")
    else:
        print("All BLAST commands completed successfully!")

# Run the function
run_cross_species_blast(main_gene_folder, db_folder, output_folder, evalue, max_hits)


Processing gene: sosie
Running BLASTp: Drosophila_melanogaster vs Drosophila_teissieri using blast_databases/sosie/Drosophila_teissieri_combined


/var/folders/mj/yrtz1pp53zs_hqrl2_mcw9m80000gn/T/ipykernel_11450/4042733987.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  query_results_df = pd.concat([query_results_df, result_df], ignore_index=True)


Running BLASTp: Drosophila_melanogaster vs Drosophila_yakuba using blast_databases/sosie/Drosophila_yakuba_combined
Running BLASTp: Drosophila_melanogaster vs Drosophila_ananassae using blast_databases/sosie/Drosophila_ananassae_combined
Running BLASTp: Drosophila_melanogaster vs Drosophila_erecta using blast_databases/sosie/Drosophila_erecta_combined
Running BLASTp: Drosophila_melanogaster vs Drosophila_simulans using blast_databases/sosie/Drosophila_simulans_combined
Running BLASTp: Drosophila_melanogaster vs Drosophila_mauritiana using blast_databases/sosie/Drosophila_mauritiana_combined
Results for Drosophila_melanogaster vs all saved to: cross_species_blast_results/sosie/Drosophila_melanogaster_vs_all.csv
Running BLASTp: Drosophila_teissieri vs Drosophila_melanogaster using blast_databases/sosie/Drosophila_melanogaster_combined
Running BLASTp: Drosophila_teissieri vs Drosophila_yakuba using blast_databases/sosie/Drosophila_yakuba_combined
Running BLASTp: Drosophila_teissieri vs Dr

/var/folders/mj/yrtz1pp53zs_hqrl2_mcw9m80000gn/T/ipykernel_11450/4042733987.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master_df = pd.concat([master_df, query_results_df], ignore_index=True)
/var/folders/mj/yrtz1pp53zs_hqrl2_mcw9m80000gn/T/ipykernel_11450/4042733987.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  query_results_df = pd.concat([query_results_df, result_df], ignore_index=True)


Running BLASTp: Drosophila_teissieri vs Drosophila_erecta using blast_databases/sosie/Drosophila_erecta_combined
Running BLASTp: Drosophila_teissieri vs Drosophila_simulans using blast_databases/sosie/Drosophila_simulans_combined
Running BLASTp: Drosophila_teissieri vs Drosophila_mauritiana using blast_databases/sosie/Drosophila_mauritiana_combined
Results for Drosophila_teissieri vs all saved to: cross_species_blast_results/sosie/Drosophila_teissieri_vs_all.csv
Running BLASTp: Drosophila_yakuba vs Drosophila_melanogaster using blast_databases/sosie/Drosophila_melanogaster_combined
Running BLASTp: Drosophila_yakuba vs Drosophila_teissieri using blast_databases/sosie/Drosophila_teissieri_combined
Running BLASTp: Drosophila_yakuba vs Drosophila_ananassae using blast_databases/sosie/Drosophila_ananassae_combined


...

Running BLASTp: Drosophila_erecta vs Drosophila_yakuba using blast_databases/scaf_(scarface)/Drosophila_yakuba_combined
Running BLASTp: Drosophila_erecta vs Drosophila_ananassae using blast_databases/scaf_(scarface)/Drosophila_ananassae_combined
Running BLASTp: Drosophila_erecta vs Drosophila_simulans using blast_databases/scaf_(scarface)/Drosophila_simulans_combined
Running BLASTp: Drosophila_erecta vs Drosophila_mauritiana using blast_databases/scaf_(scarface)/Drosophila_mauritiana_combined
Results for Drosophila_erecta vs all saved to: cross_species_blast_results/scaf_(scarface)/Drosophila_erecta_vs_all.csv
Running BLASTp: Drosophila_simulans vs Drosophila_melanogaster using blast_databases/scaf_(scarface)/Drosophila_melanogaster_combined
Running BLASTp: Drosophila_simulans vs Drosophila_teissieri using blast_databases/scaf_(scarface)/Drosophila_teissieri_combined


/var/folders/mj/yrtz1pp53zs_hqrl2_mcw9m80000gn/T/ipykernel_11450/4042733987.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  query_results_df = pd.concat([query_results_df, result_df], ignore_index=True)


Running BLASTp: Drosophila_simulans vs Drosophila_yakuba using blast_databases/scaf_(scarface)/Drosophila_yakuba_combined
Running BLASTp: Drosophila_simulans vs Drosophila_ananassae using blast_databases/scaf_(scarface)/Drosophila_ananassae_combined
Running BLASTp: Drosophila_simulans vs Drosophila_erecta using blast_databases/scaf_(scarface)/Drosophila_erecta_combined
Running BLASTp: Drosophila_simulans vs Drosophila_mauritiana using blast_databases/scaf_(scarface)/Drosophila_mauritiana_combined
Results for Drosophila_simulans vs all saved to: cross_species_blast_results/scaf_(scarface)/Drosophila_simulans_vs_all.csv
Running BLASTp: Drosophila_mauritiana vs Drosophila_melanogaster using blast_databases/scaf_(scarface)/Drosophila_melanogaster_combined
Running BLASTp: Drosophila_mauritiana vs Drosophila_teissieri using blast_databases/scaf_(scarface)/Drosophila_teissieri_combined


/var/folders/mj/yrtz1pp53zs_hqrl2_mcw9m80000gn/T/ipykernel_11450/4042733987.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  query_results_df = pd.concat([query_results_df, result_df], ignore_index=True)


Running BLASTp: Drosophila_mauritiana vs Drosophila_yakuba using blast_databases/scaf_(scarface)/Drosophila_yakuba_combined
Running BLASTp: Drosophila_mauritiana vs Drosophila_ananassae using blast_databases/scaf_(scarface)/Drosophila_ananassae_combined
Running BLASTp: Drosophila_mauritiana vs Drosophila_erecta using blast_databases/scaf_(scarface)/Drosophila_erecta_combined
Running BLASTp: Drosophila_mauritiana vs Drosophila_simulans using blast_databases/scaf_(scarface)/Drosophila_simulans_combined
Results for Drosophila_mauritiana vs all saved to: cross_species_blast_results/scaf_(scarface)/Drosophila_mauritiana_vs_all.csv
All BLAST results saved to: cross_species_blast_results/master_blast_results.csv
